## Data cleaning

This script does the following: 

1. embed the interview transcriptions, split by question, from round 1 (both automatic transcription and ground truth) and round 2 (only automatic transcription, no ground truth available) using openai and voyageai embedding models. Transcriptions come with controls ( = administrative data on the clients) from previous processing.
2. merge transcriptions and survey data (from qualtrics)
3. concatenate round 1 and 2 and save dataset for further analysis and visualization
4. generate dataset of full transcriptions (not split by question) embedded with voyageai model specialized for retrieval tasks

Output: 
1. ../../data/transcriptions_all/transcriptions_with_controls_round1_nova2.xlsx
2. ../../data/transcriptions_all/embeddings/embeddings_openai_split_gpt-4o_nova2_withcontrols_round1-2.pkl
3. ../../data/transcriptions_all/embeddings/embeddings_openai_split_gpt-4turbo_groundtruth_withcontrols_round1.pkl
4. ../../data/transcriptions_all/embeddings/embeddings_voyage_split_gpt-4o_nova2_withcontrols_round1-2.pkl
5. ../../data/transcriptions_all/embeddings/embeddings_voyage_split_gpt-4turbo_groundtruth_withcontrols_round1.pkl
6. ../../data/transcriptions_all/embeddings/embeddings_voyage_nova2_withcontrols_round1-2.pkl

In [12]:
# import packages
%reload_ext autoreload
%autoreload 2
import pandas as pd
import utils
import os

# set directory paths
data_dir = "../../data"
transcripts_dir = os.path.join(data_dir, "transcriptions_all")
split_transcript_dir = os.path.join(data_dir, "transcriptions_all/splits")
qualtrics_dir = os.path.join(data_dir, "working/qualtrics")
output_dir = os.path.join(data_dir, "transcriptions_all/embeddings") 
embeddings_dir = output_dir 
transcriptions_round1_dir = os.path.join(transcripts_dir, "transcriptions_round1_relabeled") # full transcripts without controls

# check if output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### set file paths

# qualtrics data
qualtrics_round1_path = os.path.join(qualtrics_dir, "qualtrics_23mar.dta")
qualtrics_round2_path = os.path.join(qualtrics_dir, "qualtrics_v2_07may.dta")


# full transcripts with controls
transcriptions_round1_with_controls = os.path.join(transcripts_dir, "transcriptions_with_controls_round1_nova2.xlsx") # to be created below
transcriptions_round2_with_controls = os.path.join(transcripts_dir, "transcriptions_with_controls_round2_nova2.xlsx") # available from previous processing step

# split transcripts with controls
split_transcripts_round1 = os.path.join(split_transcript_dir, "1-shot_gpt-4o_nova2_round1.xlsx")
split_transcripts_round2 = os.path.join(split_transcript_dir, "1-shot_gpt-4o_nova2_round2.xlsx")
split_transcripts_groundtruth = os.path.join(split_transcript_dir, "1-shot_gpt-4turbo_groundtruth_round1.xlsx")


# output paths openai embeddings
output_path_nova2_openai = os.path.join(output_dir, "embeddings_openai_split_gpt-4o_nova2_withcontrols_round1-2.pkl")
output_path_groundtruth_openai = os.path.join(output_dir, "embeddings_openai_split_gpt-4turbo_groundtruth_withcontrols_round1.pkl")

# output paths voyage embeddings
# split
output_path_nova2_voyage = os.path.join(output_dir, "embeddings_voyage_split_gpt-4o_nova2_withcontrols_round1-2.pkl")
output_path_groundtruth_voyage = os.path.join(output_dir, "embeddings_voyage_split_gpt-4turbo_groundtruth_withcontrols_round1.pkl")

# full transcriptions
output_path_nova2_full_voyage = os.path.join(embeddings_dir, "embeddings_voyage_nova2_withcontrols_round1-2.pkl")



### Create datasets split by question

In [6]:
# define function to get embeddings dataset

def gen_embeddings_dataset(df, columns_to_embed, qualtrics_path, controls = ['female', 'age', 'education', 'employment', 'duration', 'total_winnings'], model_type='openai', num_questions=7):
    # initialize empty kwargs
    kwargs = {}
    
    # model selection logic
    if model_type == 'openai':
        embedding_function = utils.safe_compute_embeddings
    elif model_type == 'voyage':
        embedding_function = utils.get_embeddings_voyage 
        kwargs['input_type'] = 'classification'

    else:
        raise ValueError('model_type must be either "openai" or "voyage"')
    
    # generate embeddings
    df_split_emb = embedding_function(df, columns_to_embed=columns_to_embed, **kwargs)
    embedded_columns = [ c + '_embedding' for c in columns_to_embed]
    df_long = utils.reshape_answers_split_long(df_split_emb, embedded_columns, num_questions=num_questions)
    df_long = df_long.reset_index(drop = True)

    qualtrics = pd.read_stata(qualtrics_path)
    qualtrics['reference_phone'] = qualtrics['reference_phone'].astype(float).astype(int)

    df_long_with_controls = pd.merge(df_long, qualtrics[['reference_phone'] + controls], on='reference_phone', how='left')
    print("Number of rows in the dataset: ", len(df_long))
    return df_long_with_controls

Embedding round 1

In [3]:
# Load the Excel file
df_split = pd.read_excel(split_transcripts_round1)

# initialize list of columns to embed
columns_to_embed = [f"question_answer_{i+1}_gpt-4o" for i in range(7)]
df_split['winner_label'] = 'non-winner'
df_split.loc[df_split['winner'] == 1, 'winner_label'] = 'winner'

# embed with openai
df_long_round1_with_controls_openai = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round1_path, model_type='openai')
# rename total_winnings to amount_won
df_long_round1_with_controls_openai = df_long_round1_with_controls_openai.rename(columns = {'total_winnings':'amount_won'})

# embed with voyage
df_long_round1_with_controls_voyage = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round1_path, model_type='voyage')
# rename total_winnings to amount_won
df_long_round1_with_controls_voyage = df_long_round1_with_controls_voyage.rename(columns = {'total_winnings':'amount_won'})


Embedding question_answer_7_gpt-4o: 100%|██████████| 58/58 [00:03<00:00, 15.96it/s] 
/Users/marcocaporaletti/anaconda3/envs/voices2024/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of rows in the dataset:  328
Embedding column question_answer_1_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 11354
Embedding documents 32 to 64
Total tokens: 10373
Embedding column question_answer_2_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 12601
Embedding documents 32 to 64
Total tokens: 8666
Embedding column question_answer_3_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 8455
Embedding documents 32 to 64
Total tokens: 5706
Embedding column question_answer_4_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 9349
Embedding documents 32 to 64
Total tokens: 6471
Embedding column question_answer_5_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 7229
Embedding documents 32 to 64
Total tokens: 5232
Embedding column question_answer_6_gpt-4o. Total documents: 58
Embedding documents 0 to 32
Total tokens: 10000
Embedding documents 32 to 64
Total tokens: 8208
Embedding co

Embedding round 2

In [7]:
# Load the Excel file
df_split = pd.read_excel(split_transcripts_round2)

# initialize list of columns to embed
columns_to_embed = [f"question_answer_{i+1}_gpt-4o" for i in range(9)]
df_split['winner_label'] = 'non-winner'
df_split.loc[df_split['winner'] == 1, 'winner_label'] = 'winner'

# embed with openai
df_long_round2_with_controls_openai = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round2_path, controls =  ['female', 'age', 'education', 'employment', 'duration', 'amount_won'], model_type='openai', num_questions=9)

# embed with voyage
df_long_round2_with_controls_voyage = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round2_path, controls =  ['female', 'age', 'education', 'employment', 'duration', 'amount_won'], model_type='voyage', num_questions=9)

Embedding question_answer_9_gpt-4o: 100%|██████████| 75/75 [00:10<00:00,  7.24it/s]


Number of rows in the dataset:  489
Embedding column question_answer_1_gpt-4o. Total documents: 75
Embedding documents 0 to 32
Total tokens: 12270
Embedding documents 32 to 64
Total tokens: 11899
Embedding documents 64 to 96
Total tokens: 4290
Embedding column question_answer_2_gpt-4o. Total documents: 75
Embedding documents 0 to 32
Total tokens: 10524
Embedding documents 32 to 64
Total tokens: 10561
Embedding documents 64 to 96
Total tokens: 2556
Embedding column question_answer_3_gpt-4o. Total documents: 75
Embedding documents 0 to 32
Total tokens: 6583
Embedding documents 32 to 64
Total tokens: 5414
Embedding documents 64 to 96
Total tokens: 1593
Embedding column question_answer_4_gpt-4o. Total documents: 75
Embedding documents 0 to 32
Total tokens: 9393
Embedding documents 32 to 64
Total tokens: 8084
Embedding documents 64 to 96
Total tokens: 3312
Embedding column question_answer_5_gpt-4o. Total documents: 75
Embedding documents 0 to 32
Total tokens: 6430
Embedding documents 32 to 

Concatenate datasets

In [ ]:
# add round variable to both dataframes
df_long_round1_with_controls_openai['round'] = 1
df_long_round2_with_controls_openai['round'] = 2
df_long_round1_with_controls_voyage['round'] = 1
df_long_round2_with_controls_voyage['round'] = 2

# merge the two dataframes
df_long_openai = pd.concat([df_long_round1_with_controls_openai, df_long_round2_with_controls_openai])
df_long_openai = df_long_openai.reset_index(drop = True)
df_long_voyage = pd.concat([df_long_round1_with_controls_voyage, df_long_round2_with_controls_voyage])
df_long_voyage = df_long_voyage.reset_index(drop = True)

# add winner_label times round
df_long_openai['winner_round'] = df_long_openai['winner_label'] + '_round' + df_long_openai['round'].astype(str)
df_long_voyage['winner_round'] = df_long_voyage['winner_label'] + '_round' + df_long_voyage['round'].astype(str)

df_long_openai.drop(columns = "reference_phone").head()

In [ ]:
df_long_voyage.drop(columns = "reference_phone").head()

In [13]:
# save the final dataframes
df_long_openai.to_pickle(output_path_nova2_openai)
df_long_voyage.to_pickle(output_path_nova2_voyage)

Repeat for ground truth (round 1)

In [ ]:
# Load the Excel file
df_split = pd.read_excel(split_transcripts_groundtruth)

# initialize list of columns to embed
columns_to_embed = [f"question_answer_{i+1}_gpt-4-turbo-2024-04-09" for i in range(7)]
df_split['winner_label'] = 'non-winner'
df_split.loc[df_split['winner'] == 1, 'winner_label'] = 'winner'


# embed with openai
df_long_round1_groundtruth_with_controls_openai = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round1_path, model_type='openai')
# rename total_winnings to amount_won
df_long_round1_groundtruth_with_controls_openai = df_long_round1_with_controls_openai.rename(columns = {'total_winnings':'amount_won'})

# embed with voyage
df_long_round1_groundtruth_with_controls_voyage = gen_embeddings_dataset(df_split, columns_to_embed, qualtrics_round1_path, model_type='voyage')
# rename total_winnings to amount_won
df_long_round1_groundtruth_with_controls_voyage = df_long_round1_with_controls_voyage.rename(columns = {'total_winnings':'amount_won'})

# check heads
df_long_round1_groundtruth_with_controls_openai.drop(columns = "reference_phone").head()


In [ ]:
df_long_round1_groundtruth_with_controls_voyage.drop(columns = "reference_phone").head()

In [8]:
# reset index of the dataframes
df_long_round1_groundtruth_with_controls_openai = df_long_round1_groundtruth_with_controls_openai.reset_index(drop = True)
df_long_round1_groundtruth_with_controls_voyage = df_long_round1_groundtruth_with_controls_voyage.reset_index(drop = True)

# save the dataframes
df_long_round1_groundtruth_with_controls_openai.to_pickle(output_path_groundtruth_openai)
df_long_round1_groundtruth_with_controls_voyage.to_pickle(output_path_groundtruth_voyage)

### Create dataset of full transcription embeddings for retrieval (voyage model)

In [ ]:
out_path = transcriptions_round1_with_controls
df_round1, samples_round1 = utils.make_dataset_from_txt_and_dta(dta_path=qualtrics_round1_path, output_path=out_path, directory=transcriptions_round1_dir)
print(f"Number of transcriptions left after cleaning: {samples_round1}.\n")

In [ ]:
df_emb_round1 = utils.get_embeddings_voyage(df_round1, columns_to_embed=["transcription"], batch_size=32, input_type="document")

In [ ]:
df_round2 = pd.read_excel(transcriptions_round2_with_controls)
df_emb_round2 = utils.get_embeddings_voyage(df_round2, columns_to_embed=["transcription"], batch_size=32)

In [ ]:
# add round variable to both dataframes
df_emb_round1['round'] = 1
df_emb_round2['round'] = 2

# merge the two dataframes
df_emb_all = pd.concat([df_emb_round1, df_emb_round2], ignore_index = True)
df_emb_all.reset_index(drop=True, inplace=True)

# save the final dataframe
out_path = output_path_nova2_full_voyage
df_emb_all.to_pickle(out_path)